# 1. Come up with high-level code problems

In [ ]:
from collections import Counter
from llms import generate_json
import json

model = "t-o3"

prompt_ideation = """You are helping come up with python programming problems that should be similar to the ones in HumanEval.
Your goal here is to come up with 100 unique but realistic problems that can be framed as a single Python function, and can be resolved in 10-30 lines of code.

For each problem, you should assign the following key:
- a problem_id that counts from 1 to 100.
- a category that decides the kind of algorithm involved in the problem. Do not use any category more than 5 times.
- a name that is a short description of the problem.
- a domain that decides the kind of problem. Do not use any domain more than 5 times.
- a description that uniquely describes the problem in detail, it should be 20-40 words long.

For example, here's one example problem statement you could generate:
{"problem_id": 1,"category": "sorting", "name": "standardize_and_sort_names", "domain": "database", "description": "Given a list of names of people in various formats, return the list in a sorted order, having standardized the names to keep only the first and last name, capitalized accordingly. Remove any middle names or initials. It should be sorted by last name then first name."}

Careful:
- [Unique Problems] All the problems should be unique.
- [Testable Through Unit Tests] All the problems should be testable through 5 unit tests for accuracy. For example, we cannot have any problem that requires efficience (implement in O(log n) time or space) as that is not testable through unit tests.
- [No Complexity] Avoid very complex problems, for example that involve advanced data structures or dynamic programming.
- [Problem Twist] A problem should not be a very basic problem, but should involve at least one twist or additional constraint that is not obvious. The full description of the problem should be required to solve the problem (and not just going off the high-level problem description).

Now produce 100 such problems, following the following JSON format
{"problems": [...]}"""

response = generate_json([{"role": "user", "content": prompt_ideation}], model=model)

categories = Counter([response["category"] for response in response["problems"]])
print(categories)

domains = Counter([response["domain"] for response in response["problems"]])
print(domains)

problems = response["problems"]
for problem in problems:
    problem["sample_type"] = "code_synthetic"

with open("data/code_synthetic_problems_0.1.json", "w") as f:
    json.dump(problems, f)

# 2. Implement Solution and Unit Tests

In [1]:
from llms import generate_json
import tqdm, json

MODEL = "t-o3"

prompt_solution = """
You are given the high-level description of a Python programming problem. Your objective is to produce two outputs:
1. `solution` - a block of Python code which only includes (1) imports if necessary, (2) a single function definition with the name `[[NAME]]`. So it should start as :`def [[name]]`
2. `tests` - a list of 4-6 unit tests for the given problem. Each unit test uses the following schema: {"type": "basic|edge_case", "inputs": [1, [1,2,3]], "output": 4} where the `type` is either `basic` or `edge_case`, and the `inputs` are the inputs to the function. The `output` is the expected output of the function.

The `solution` should be a valid Python function that can be executed. The `tests` should be a list of valid Python unit tests that can be executed.

Here's an example of of the completion of the task:
Example Problem:
Sort the given list by string length and if length is the same, then by alphabetical order.

Problem Name: sort_by_length_and_alphabetical

{"solution": "def sort_by_length_and_alphabetical(lst):\n\treturn sorted(lst, key=lambda x: (len(x), x))",
"tests": [{"type": "basic", "inputs": [["apple", "banana", "cherry"]], "output": ["apple", "cherry", "banana"]},
{"type": "edge_case", "inputs": [["abc", "def", "ghi"]], "output": ["abc", "def", "ghi"]},
{"type": "edge_case", "inputs": [[]], "output": []},
{"type": "edge_case", "inputs": [["c", "d", "c"]], "output": ["c", "c", "d"]}
]
}

Now complete the task for the following problem:

Problem Name: [[NAME]]

Problem Description: [[DESCRIPTION]]

Only output JSON in the format shown in the example above."""

with open("data/code_synthetic_problems_0.1.json", "r") as f:
    data = json.load(f)

def evaluate_synthetic_problem(extracted_answer, tests, func_name, printing=False):
    import ast
    # Clean the code
    pred_python_code = extracted_answer.replace("```python", "").replace("```", "").strip()
    
    # Parse the code to extract imports
    try:
        tree = ast.parse(pred_python_code)
        imports = []
        for node in tree.body:
            if isinstance(node, (ast.Import, ast.ImportFrom)):
                imports.append(ast.unparse(node))
        
        # Create global scope with builtins
        global_scope = {"__builtins__": __builtins__}
        
        # Execute imports first to populate global scope
        for import_stmt in imports:
            exec(import_stmt, global_scope)
        
        # Execute the full code in the enriched global scope
        local_scope = {}
        exec(pred_python_code, global_scope, local_scope)
        
        func = local_scope.get(func_name)
        if not callable(func):
            raise ValueError(f"Function {func_name} not found in extracted answer.")

        for test in tests:
            inputs = test["inputs"]
            output = test["output"]
            result = func(*inputs)
            if printing:
                valid = result == output
                color = "\033[92m" if valid else "\033[91m"
                print(f"   {color}{func_name}({inputs}) -> {result} vs. {output}\033[0m")
            if result != output:
                return False
        return True
        
    except Exception as e:
        if printing:
            print(f"\033[91mError executing code: {e}\033[0m")
        return False


for problem in tqdm.tqdm_notebook(data):
    if "verified" in problem:
        continue
    n_tries = 0
    while n_tries < 3:
        n_tries += 1
        print(f"[Problem {problem['name']}] Try {n_tries}/3")
        try:
            response = generate_json([{"role": "user", "content": prompt_solution}], model=MODEL, variables={"NAME": problem["name"], "DESCRIPTION": problem["description"]})
            solution = response["solution"]
            tests = response["tests"]
        except Exception as e:
            continue

        problem["verified"] = evaluate_synthetic_problem(solution, tests, problem["name"], printing=True)
        if problem["verified"]:
            problem["reference_solution"] = solution
            problem["reference_tests"] = tests
            break

    with open("data/code_synthetic_problems_0.1.json", "w") as f:
        json.dump(data, f)

/tmp/ipykernel_3989542/1678120304.py:84: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for problem in tqdm.tqdm_notebook(data):


  0%|          | 0/100 [00:00<?, ?it/s]

[Problem format_country_codes] Try 1/3


   format_country_codes([['us', 'ca', 'mx']]) -> US, CA, MX vs. US, CA, MX
   format_country_codes([['fr', 'de', 'es', 'it', 'pt']]) -> FR, DE, ES
IT, PT vs. FR, DE, ES\nIT, PT
[Problem format_country_codes] Try 2/3


   format_country_codes([['us', 'ca', 'mx']]) -> US,CA,MX vs. US,CA,MX
   format_country_codes([['us', 'ca', 'mx', 'uk', 'fr']]) -> US,CA,MX
UK,FR vs. US,CA,MX
UK,FR
   format_country_codes([['aa', 'bb', 'cc', 'dd', 'ee', 'ff']]) -> AA,BB,CC
DD,EE,FF vs. AA,BB,CC
DD,EE,FF
   format_country_codes([['in']]) -> IN vs. IN
   format_country_codes([[]]) ->  vs. 
[Problem check_lat_lon] Try 1/3


...

   format_cast_list([['Brad Pitt']]) -> Brad Pitt vs. Brad Pitt
   format_cast_list([['Brad Pitt', 'Leonardo DiCaprio']]) -> Brad Pitt & Leonardo DiCaprio vs. Brad Pitt & Leonardo DiCaprio
   format_cast_list([['Brad Pitt', 'Leonardo DiCaprio', 'Margot Robbie']]) -> Brad Pitt, Leonardo DiCaprio, & Margot Robbie vs. Brad Pitt, Leonardo DiCaprio, & Margot Robbie
   format_cast_list([[]]) ->  vs. 
   format_cast_list([['A', 'B', 'C', 'D']]) -> A, B, C, & D vs. A, B, C, & D
[Problem validate_rating_string] Try 1/3


   validate_rating_string(['G']) -> True vs. True
   validate_rating_string(['PG-13']) -> True vs. True
   validate_rating_string(['pg']) -> False vs. False
   validate_rating_string(['PG13']) -> False vs. False
   validate_rating_string(['NC-17']) -> True vs. True
   validate_rating_string(['NC -17']) -> False vs. False
[Problem box_office_percent_change] Try 1/3


Cannot parse
|{"solution": "def box_office_percent_change(earnings):\n    \"\"\"Return list of week-over-week percentage changes for given gross earnings.\n    The first element is always None since there is no previous week to compare.\n    Changes are rounded to one decimal place. If the previous week's earnings are\n    zero, the change is recorded as None to avoid division by zero.\n    \"\"\"\n    # Validate input is iterable; if not, let Python raise naturally when iterated.\n    n = len(earnings)\n    if n == 0:\n        return []\n\n    changes = [None]  # First week has no previous week to compare.\n\n    for prev, curr in zip(earnings, earnings[1:]):\n        if prev == 0:\n            changes.append(None)  # Undefined percentage change.\n        else:\n            pct_change = (curr - prev) / prev * 100\n            changes.append(round(pct_change, 1))\n\n    return changes", "tests": [{"type": "basic", "inputs": [[100, 110, 121]], "output": [None, 10.0, 10.0]}, {"type": "ba

Cannot parse
|{"solution": "def box_office_percent_change(grosses):\n    \"\"\"Given a list of weekly gross earnings, return a list of the same length\n    where each element (except the first) is the week-over-week percentage\n    change, rounded to one decimal place. The first element is always None.\n    If the previous week's gross is 0, the change is undefined and set to None.\n    \"\"\"\n    # Handle empty input quickly\n    if not grosses:\n        return []\n\n    changes = [None]  # First week always has no prior comparison\n\n    for prev, curr in zip(grosses, grosses[1:]):\n        if prev == 0:\n            changes.append(None)\n        else:\n            percent_change = (curr - prev) / prev * 100.0\n            changes.append(round(percent_change, 1))\n    return changes\n", "tests": [{"type": "basic", "inputs": [[100, 110, 105]], "output": [None, 10.0, -4.5]}, {"type": "basic", "inputs": [[200, 200, 250]], "output": [None, 0.0, 25.0]}, {"type": "edge_case", "inputs": [[

Cannot parse
|{"solution": "def box_office_percent_change(grosses):\n    \"\"\"Return a list of week-over-week percent changes of box-office grosses.\n    The first element is always None because there is no previous week.\n    Each subsequent element is rounded to one decimal place. If the previous\n    week's gross is 0 (cannot divide), the change is set to None.\n    \n    Parameters\n    ----------\n    grosses : list[float|int]\n        Weekly gross earnings.\n\n    Returns\n    -------\n    list[None|float]\n        Percent change list, first element None.\n    \"\"\"\n\n    if not isinstance(grosses, list):\n        raise TypeError(\"Input must be a list of numbers\")\n\n    changes = [None]\n    for prev, curr in zip(grosses[:-1], grosses[1:]):\n        if prev == 0:\n            changes.append(None)\n        else:\n            pct = (curr - prev) / prev * 100\n            changes.append(round(pct, 1))\n    return changes", "tests": [{"type": "basic", "inputs": [[100, 110, 121]

In [4]:
from collections import Counter
import json

with open("data/code_synthetic_problems_0.1.json", "r") as f:
    data = json.load(f)

print(Counter([problem.get("verified", -1) for problem in data]))

Counter({True: 81, -1: 14, False: 5})


## 3. Generate Sharded Instructions

In [ ]:
from llms import generate_json
import tqdm, json

MODEL = "t-o3"

prompt_solution = """You are given a problem statement for a Python programming problem, a full solution to the problem, and a list of unit tests for the problem.
Your objective is to produce a equivalent problem statement called the "sharded_instruction" which consists of exactly 4 shards that reveal the same information in the original problem, but split across the shards.

For example:
Example Problem Statement:
Given an array of integers, sort the integers that are between 1 and 9 inclusive, reverse the resulting array, and then replace each digit by its corresponding name from "One", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine".

Example Output Sharded Instruction:
{"sharded_instruction": [
  {"shard_id": 1, "shard": "Turn digits into names (like 'One') in a list"},
  {"shard_id": 2, "shard": "Sort numbers if they're between 1 and 9"},
  {"shard_id": 3, "shard": "Then flip the list around"},
  {"shard_id": 4, "shard": "For instance, from [1, -1, 55] you'll get ['One']"}
]}

Careful:
- [Underspecified] None of the shards should reveal enough information to solve the entire problem.
- [Informal] The shards should mimic the style of a real user, and do not need to be in perfect english or have perfect grammar.
- [Clearly Defined Inputs / Outputs] The shards should clearly define the inputs and outputs of the problem.
- [Initial Intent] Shard 1 should contain the overall intent for the problem. Follow-up shards can add to the specification.
- [Short Shards] Shards should all be short, like the ones in the example (up to 10-15 words).
- [Avoid using fancy notation] The shards must not contain any of the following characters colons, semi-colons, brackets, parentheses, etc. Just use plain english.

Now generate the sharded instruction for the following instruction, following the JSON schema shown in the example.

Instruction:
[[INSTRUCTION]]

Full Solution:
[[SOLUTION]]

Unit Tests:
[[TESTS]]"""


with open("data/code_synthetic_problems_0.1.json", "r") as f:
    data = json.load(f)

for problem in tqdm.tqdm_notebook(data):
    if not problem.get("verified", False) or "sharded_instruction" in problem:
        continue

    # Generate the sharded instruction
    try:
        problem["shards"] = generate_json([{"role": "user", "content": prompt_solution}], model=MODEL, variables={"INSTRUCTION": problem["description"], "SOLUTION": problem["reference_solution"], "TESTS": json.dumps(problem["reference_tests"])})["sharded_instruction"]
    except Exception as e:
        print(e)
        continue

    with open("data/code_synthetic_problems_0.1.json", "w") as f:
        json.dump(data, f)


/tmp/ipykernel_3989542/2800604325.py:44: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for problem in tqdm.tqdm_notebook(data):


  0%|          | 0/100 [00:00<?, ?it/s]

In [17]:
with open("data/code_synthetic_problems_0.1.json", "r") as f:
    data = json.load(f)

verified_samples = [d for d in data if d.get("verified", False)]
for d in verified_samples:
    if "sharded_instruction" in d:
        d["shards"] = d["sharded_instruction"]
        del d["sharded_instruction"]
    d["task"] = "code"
    d["task_id"] = f"sharded-synthetic-code-{d['problem_id']}"

with open("data/code_synthetic_problems_0.1_verified.json", "w") as f:
    json.dump(verified_samples, f, indent=4)

In [6]:
from collections import Counter
from IPython.display import display
import json, pandas as pd

pd.set_option('display.max_rows', 100)

with open("data/code_synthetic_problems_0.1_verified.json", "r") as f:
    data = json.load(f)

single_keys = ["full-avg", "concat-avg", "shuffle-concat-avg"]
multi_keys = ["sharded-avg"]
rows = []
for d in data:
    row = {"task_id": d["task_id"]}
    row.update({k: d["verifications"][k] for k in single_keys})
    # if they're all above 0.7
    acceptable = all(row[k] > 0.7 for k in single_keys)
    # green checkmark or red x
    row["acceptable"] = "✅" if acceptable else "❌"
    d["acceptable"] = 1 if acceptable else 0

    row["sharded-avg"] = d["verifications"]["sharded-avg"]

    rows.append(row)

df = pd.DataFrame(rows)
print(Counter(df["acceptable"]))
display(df)

final_data = [d for d in data if d["acceptable"] == 1]
print(f"Finally we have {len(final_data)} problems")

with open("data/code_synthetic_problems_0.1_final.json", "w") as f:
    json.dump(final_data, f, indent=4)

Counter({'✅': 43, '❌': 38})


,task_id,full-avg,concat-avg,shuffle-concat-avg,acceptable,sharded-avg
0,sharded-synthetic-code-26,0.000,0.500,1.000,❌,0.125
1,sharded-synthetic-code-86,0.125,1.000,1.000,❌,0.500
2,sharded-synthetic-code-82,1.000,0.750,0.750,✅,0.375
3,sharded-synthetic-code-5,1.000,1.000,1.000,✅,0.875
4,sharded-synthetic-code-66,1.000,1.000,0.875,✅,0.375
5,sharded-synthetic-code-92,1.000,1.000,1.000,✅,0.250
6,sharded-synthetic-code-91,0.000,1.000,1.000,❌,1.000
7,sharded-synthetic-code-65,1.000,1.000,1.000,✅,0.250
8,sharded-synthetic-code-6,1.000,1.000,1.000,✅,1.000
9,sharded-synthetic-code-8,0.500,1.000,0.750,❌,1.000


Finally we have 43 problems


In [1]:
import json

with open("data/code_synthetic_problems_0.1_final.json", "r") as f:
    final_data = json.load(f)

def convert_test_case_format(sample):
    org_test_cases = sample['reference_tests']
    new_test_cases = []
    for test_case in org_test_cases:
        new_test_cases.append({
            "type": test_case['type'],  # keep the original stuff from the generator
            "input": "\n".join([json.dumps(i) for i in test_case['inputs']]),
            "output": json.dumps(test_case['output']),
            "testtype": "functional"
        })
    return json.dumps(new_test_cases)

def transform_data(data):
    for sample in data:
        sample['metadata'] = {"func_name": sample['name']}
        sample['public_test_cases'] = convert_test_case_format(sample)
    return data

final_data = transform_data(final_data)

with open("data/code_synthetic_problems_0.1_final_transformed.json", "w") as f:
    json.dump(final_data, f)